# Introduction
Text generation is another ability of RNN. In this part, we will find out how to create a generative model for text. In detail, a LSTM network which we will build learn sequences of characters from Alice in Wonderland.


Dataset: we choose a dataset from familiar novel [Alice’s Adventures in Wonderland](http://www.gutenberg.org/cache/epub/11/pg11.txt) by Lewis Carroll



## Importing the classes and functions

In [30]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys

## Load dataset

In [36]:
filename = "pg11.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
#convert all of the characters to lowercase to reduce the vocabulary that the network must learn

## Convert into integer


In [37]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

print("chars")
print(chars)
print("char_to_int")
print(char_to_int)

chars
['\n', '\r', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', ':', ';', '?', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
char_to_int
{'\n': 0, '\r': 1, '!': 3, ' ': 2, '"': 4, "'": 5, ')': 7, '(': 6, '*': 8, '-': 10, ',': 9, '.': 11, ';': 13, ':': 12, '?': 14, '_': 15, 'a': 16, 'c': 18, 'b': 17, 'e': 20, 'd': 19, 'g': 22, 'f': 21, 'i': 24, 'h': 23, 'k': 26, 'j': 25, 'm': 28, 'l': 27, 'o': 30, 'n': 29, 'q': 32, 'p': 31, 's': 34, 'r': 33, 'u': 36, 't': 35, 'w': 38, 'v': 37, 'y': 40, 'x': 39, 'z': 41}


## Check size of dataset


In [38]:
n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab

Total Characters:  28851
Total Vocab:  42


## Create data for training 

We will split the book text up into subsequences with a fixed length(here we choose 100 character). Each training pattern of the network is comprised of 100 time steps of one character (X) followed by one character output (y). When creating these sequences, we slide this window along the whole book one character at a time. 
Let see a simple example with sequence's length is 5
- CHAPT -> E
- HAPTE -> R

In [39]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

Total Patterns:  28751


## Prepare data for training 
- Tranform the list of input into the form expected by an LTSM network. 
- As usual, we rescale the number into range 0 to 1 to easy training network.
- Convert the output parterns into one hot encoding which we can configure the network to predict the probalities of 62 characters. Each y value is converted into a sparse vector with a length of 62, full of zeros except with a 1 in the column for the letter (integer) that the pattern represents. For example: 
(
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
)


In [40]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

## Create model 

In [41]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

## Training created model 

In [47]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(X, y, epochs=10, batch_size=128, callbacks=callbacks_list)

Epoch 1/10
28751/28751 [==============================] - 370s - loss: 2.9479   
Epoch 2/10
28751/28751 [==============================] - 371s - loss: 2.8759   
Epoch 3/10
28751/28751 [==============================] - 366s - loss: 2.8231   
Epoch 4/10
28751/28751 [==============================] - 374s - loss: 2.7931   
Epoch 5/10
28751/28751 [==============================] - 370s - loss: 2.7649   
Epoch 6/10
28751/28751 [==============================] - 369s - loss: 2.7395   
Epoch 7/10
28751/28751 [==============================] - 373s - loss: 2.7172   
Epoch 8/10
28751/28751 [==============================] - 380s - loss: 2.6907   
Epoch 9/10
28751/28751 [==============================] - 376s - loss: 2.6658   
Epoch 10/10
28751/28751 [==============================] - 370s - loss: 2.6427   


## Test

In [49]:
# create input data
int_to_char = dict((i, c) for i, c in enumerate(chars))
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""

Seed:
" you didn't like cats.'

'not like cats!' cried the mouse, in a shrill, passionate voice. 'would
y "


In [59]:
#Generate text using trained model
for i in range(50):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."

t to the wou to 
aar toe tou to the woet 
aad sou 
Done.
